## הגדרות ומקדים

### ספריות

In [2]:
import os
import sys
import fiona
import pandas as pd
import geopandas as gpd
from openpyxl import load_workbook

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [4]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

output_location=df_inputs_outputs['location'][2]

sys.path.append(software_root_folder)

## פונקציות

### פונקציות גלובליות

In [5]:
from global_functions import up_load_shp, up_load_df, drop_geo, make_point, up_load_gdb, change_cols_names

## להריץ תלמידים

In [6]:
%run "students_current_year.ipynb"

## להעלות שכבות

In [7]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_folder_location)

EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [8]:
EMP_kibolet=EMP_kibolet.fillna(0)

EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [9]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_folder_location))

In [10]:
#העלה של נתוני אנשים התפלגות גילים שנוצר לפני הקוד הזה
path=r'{}\Intermediates'.format(software_folder_location)
taz_with_pop=up_load_df(path,'taz_with_pop_info')

In [11]:
col_needed=['Taz_num','aprt_20',
 'pop',
 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up', 'hh_size']

In [12]:
taz=taz.merge(taz_with_pop[col_needed],on='Taz_num',how='left')

In [13]:
taz=taz.set_index('Taz_num')

In [14]:
taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc='sum')


In [15]:

taz=taz.fillna(0)

taz['pop_without_dorms_yeshiva']=taz['pop']

taz['pop']=taz['pop']+taz['student_dorms']

taz['pop_20_just_from_aprt']=taz['pop_20']

taz['pop_25_just_from_aprt']=taz['pop_25']

taz['pop_20']=taz['pop_20']+taz['student_dorms']*0.6

taz['pop_25']=taz['pop_25']+taz['student_dorms']*0.4

In [16]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,CR_PNIM,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [17]:
taz['pop_15_just_from_aprt']=taz['pop_15']

In [18]:

taz.loc[taz['main_secto']=="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

In [19]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,CR_PNIM,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check,pop_15_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## כימות מועסקים במרחב צתאל

In [20]:
pre_woman=0.5

pre_man=1-pre_woman

In [21]:
work_age=[ 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60']

under_work_age=[ 'pop_15', 'pop_20']

over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [22]:
taz['work_age']=taz[work_age].sum(axis=1)

taz['under_work_age']=taz[under_work_age].sum(axis=1)

taz['over_work_age']=taz[over_work_age].sum(axis=1)

In [23]:
sector='U_Orthodox'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.75+taz['work_age']*pre_man*0.55
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.07+taz['under_work_age']*pre_man*0.09
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='Jewish'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.9+taz['work_age']*pre_man*0.92
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.20+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.2+taz['over_work_age']*pre_man*0.15

sector='Arab'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='arabs_behined_seperation_wall'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09


## חישוב אבטלה

In [24]:
taz['pop_emp_employed']=0

taz.loc[taz['main_secto']=="U_Orthodox",'pop_emp_employed']=taz['pop_emp']*0.97

taz.loc[taz['main_secto']=="Jewish",'pop_emp_employed']=taz['pop_emp']*0.98

taz.loc[taz['jew']==0,'pop_emp_employed']=taz['pop_emp']*0.95

## חישוב יוממות

In [25]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_folder_location))

In [26]:
taz['commuting']=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

round(emp_left_jtmt_area,-3)

94000.0

In [27]:
emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

emp_in_jtmt_area=emp_from_jtmt_area*1.07 #יוממות פנימה מחוץ למרחב

round(emp_in_jtmt_area,-3)

510000.0

In [28]:
emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.95 #הפחחת עובדים ניידים

round(emp_in_jtmt_area_without_mobile,-3)

484000.0

In [29]:
emp_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_without_palestin_not_okev

## פיזור עוקב

In [30]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('kayim_emp>0')

EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

taz['taz_area']=taz.area

In [31]:
emp_buffer_taz=gpd.overlay(EMP_buffer,taz.reset_index())

emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

In [32]:
taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

In [33]:
taz['okev']=0
taz.loc[(~taz.index.isin(taz_num_no_need_okev))&(taz['pop']>0)&(taz['main_secto']!="Palestinia"),'okev']=1

taz.loc[taz['main_secto']=="arabs_behined_seperation_wall",'okev']=1

In [34]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

## מקדם עוקב

In [35]:
okev_level=0.15

arab_city=taz.query('jew==0 & okev==1')['aprt_20'].sum().item()

arab_sub=0

ou_sub=taz.query('main_secto=="U_Orthodox" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [36]:
ou_city=taz.query('main_secto=="U_Orthodox" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

general_city=taz.query('main_secto=="Jewish" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

general_sub=taz.query('main_secto=="Jewish" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

base_okev_factor=(emp_okev-(okev_level*general_city-okev_level*(ou_sub+arab_city)-2*okev_level*arab_sub))/(general_sub+general_city+ou_city+ou_sub+arab_city+arab_sub)

base_okev_factor=round(base_okev_factor,3)

In [37]:
data = [['Jewish', base_okev_factor,base_okev_factor+okev_level],['arab', base_okev_factor-okev_level,base_okev_factor], ['U_Orthodox',base_okev_factor-2*okev_level,base_okev_factor-okev_level]]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['sector', 'sub','city'])
okev_factor=df.set_index('sector')
okev_factor.to_excel(r'{}\Intermediates\okev_factor.xlsx'.format(software_folder_location))

In [38]:
taz['emp_okev']=0

taz.loc[(taz['jew']==0)&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['arab','city']

taz.loc[(taz['main_secto']=="U_Orthodox")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','sub']

taz.loc[(taz['main_secto']=="U_Orthodox")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','city']

taz.loc[(taz['main_secto']=="Jewish")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','city']

taz.loc[(taz['main_secto']=="Jewish")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','sub']

taz.emp_okev.sum().item()-emp_okev

-1640.507337908377

## פיצול שכבת לא עוקב לאזורי תנועה

In [39]:
EMP_kibolet['emp_area']=EMP_kibolet.area

EMP_kibolet['ID']=EMP_kibolet.index

EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

C:\Users\gidon\AppData\Local\Temp\ipykernel_30372\1806380584.py:5: UserWarning: `keep_geom_type=True` in overlay resulted in 279 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)


In [40]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')


In [41]:
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID')['small_area'].sum()

EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc='sum')[['emp_without_palestin_in_taz']]



In [42]:
taz=taz.fillna(0)

taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']

taz.query('main_secto!="Palestinian"')['total_emp'].sum()-emp_in_jtmt_area_without_mobile

-1640.5073379084934

## התפלגות ענפי תעסוקה

In [43]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_folder_location)).fillna(0)

In [44]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

emp_category_type[col]=emp_category_type[col]/100

emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

In [45]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']
taz=taz.join(emp_category_type[col]).fillna(0)

In [46]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [47]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

taz.query('check_emp_sum>10 |check_emp_sum<-10')

,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,CR_PNIM,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,SEA1,SEA2,SEA3,UOA1,UOA2,UOA3,ARA1,ARA2,ARA3,student_chardi_not_gov,student_arab_not_gov,student,Univ_AR,Univ_SE,Univ_UO,emp_uni,univ,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,Yeshiva,Seminar_dorms_pop_15,Seminar_dorms_pop_20,Seminar_dorms_pop_25,Seminar_dorms_pop_sum,emp_from_Seminar_student,Seminar,kollim_demand,add_from_kollim_demand,student_yeshiva_and_kollim,UO_Hi_Ed,emp_UO_Hi_Ed,jew,emp_from_student,emp_Education,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,hh_size,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_check,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public,check_emp_sum
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת סטודנטים אל כמות משקי בית

In [48]:
taz=taz.fillna(0)

taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [49]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [50]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [51]:
path=r'{}\background_files'.format(software_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [52]:

taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

In [59]:
save_taz_path=r'{}\Intermediates'.format(software_folder_location)
taz.to_excel(r'{}\taz_before_add_geo.xlsx'.format(save_taz_path))


## ייצוא לבקרה

In [53]:
taz=taz.reset_index()

In [54]:
pre_yeshiva_from_20_30_age=taz[['Yeshiva']].sum().sum()/taz.query('jew==1')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

pre_yeshiva_from_20_30_age

0.4971940640095134

In [55]:
pre_uni_from_20_30_age=taz[['univ']].sum().sum()/taz.query('main_secto!="Palestinian"')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

pre_uni_from_20_30_age

0.3034760359349936

## ייצוא בפורמט של מודלים

הפעולה הבאה היא להוסיף אלמנטים גיאוגרפים שהם צריכים בשביל להריץ כמו אזורי חינוך וכו' וכו' שאני חושב שהכי נכון יהיה שזה יהיה בנפרד מהקוד הזה

In [252]:
taz['yosh']=0

taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

taz['jerusalem_city']=0

taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

taz=taz.reset_index()

In [253]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

taz=change_cols_names(taz, col, col_new_name)

In [254]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(output_location,file_date),index=False)

In [255]:
#load excel file
workbook = load_workbook(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

#open workbook
sheet = workbook.active

#modify the desired cell
sheet["B4"] = forecast_version_folder_location
sheet["B5"] = False
sheet["B6"] = r'{}\current\background_files\TAZ_V4_230518_Published.shp'.format(software_root_folder)

#save the file
workbook.save(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

## בקרות לוגיות

In [256]:
taz.loc[taz['TAZ']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [257]:
taz.yosh.unique()

array([0, 1], dtype=int64)

In [258]:
taz.in_jerusalem_metropolin.unique()

array([1., 0.])

In [259]:
taz.jerusalem_city.unique()

array([1, 0], dtype=int64)

In [260]:
taz.sector.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [261]:
col=['hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [262]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])

In [263]:
taz_to_check

[]

In [264]:
taz.loc[~(taz['hh_total']<=taz['pop'])]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [265]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [266]:
col=['age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

In [267]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0].query('TAZ==511')#.to_excel('delet.xlsx')#[col].sum(axis=1)

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [268]:
len(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0])

0

In [269]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [270]:
col=[
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up']

In [271]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])

In [272]:
taz_to_check

[]

In [273]:
col=['indus',
'com_hotel',
'business',
'public',
'education',
'agri']

In [274]:
taz['emp_check']=round(taz[col].sum(axis=1)-taz['emp_tot'])

In [275]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['emp_check']!=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check


In [276]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

In [277]:
taz_to_check

[]

In [278]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check
